In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "../data/dogscats/"
sz=224

In [ ]:
??torch.multinomial

In [5]:
class WeightedRandomSampler(object):
    r"""Samples elements from [0,..,len(weights)-1] with given probabilities (weights).

    Arguments:
        weights (list)   : a list of weights, not necessary summing up to one
        num_samples (int): number of samples to draw
        replacement (bool): if ``True``, samples are drawn with replacement.
            If not, they are drawn without replacement, which means that when a
            sample index is drawn for a row, it cannot be drawn again for that row.
    """

    def __init__(self, weights, num_samples, replacement=False):
        self.weights = T(weights)
        self.num_samples = num_samples
        self.replacement = replacement

    def __iter__(self):
        return iter(torch.multinomial(self.weights, self.num_samples, self.replacement))

    def __len__(self):
        return self.num_samples

In [ ]:
wsamp = WeightedRandomSampler([0.25, 0.75], 1, replacement=False)

n0 = 0
n1 = 0
tot = 0
for t in range(1000):
    for w in wsamp:
        if w == 0:
            n0 += 1
        elif w == 1:
            n1 += 1
        tot += 1
assert tot == n0 + n1
print(n0/tot)
print(n1/tot)

In [6]:
def check_ws(ws, ds):
    n0 = 0
    n1 = 0
    tot = 0

    for w in tqdm(ws):
        d = ds[w]
        d1 = d[1]
        if d1 == 0:
            n0 += 1
        elif d1 == 1:
            n1 += 1
        tot += 1
    assert tot == n0 + n1
    print(n0/tot)
    print(n1/tot)

In [7]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), bs=16)

In [ ]:
y.shape[0]

In [ ]:
c0 = 0
c1 = 0
tot = 0
for x, y in tqdm(data.trn_dl):
    ys = y.shape[0]
    num1 = torch.sum(y)
    c0 += ys - num1
    c1 += num1
    tot += ys
assert tot == c0 + c1
print(c0 / tot)
print(c1 / tot)

In [9]:
ds = data.trn_dl.dataset

In [8]:
def check_sampler(dl):
    c0 = 0
    c1 = 0
    tot = 0
    for x, y in tqdm(dl):
        ys = y.shape[0]
        num1 = torch.sum(y)
        c0 += ys - num1
        c1 += num1
        tot += ys
    assert tot == c0 + c1
    print(c0 / tot)
    print(c1 / tot)

In [10]:
check_sampler(data.trn_dl)

100%|██████████| 1438/1438 [00:17<00:00, 81.08it/s]
0.5
0.5


In [ ]:
len(ds)

In [11]:
wl2i = []

l2p = [0.25, 0.75]

for ind, d in enumerate(tqdm(ds)):
    wl2i.append(l2p[d[1]])

100%|██████████| 23000/23000 [01:21<00:00, 281.74it/s]


In [12]:
ws2 = WeightedRandomSampler(wl2i, len(ds)//2, replacement=True)

In [13]:
check_ws(ws2, ds)

100%|██████████| 11500/11500 [00:41<00:00, 275.24it/s]
0.25121739130434784
0.7487826086956522


In [ ]:
list(ws2)

In [ ]:
a1 = ws2

In [14]:
new_dl = DataLoader(ds, batch_size=16, 
                    sampler=WeightedRandomSampler(wl2i, len(ds)//2, replacement=True),
                    num_workers=4)

In [15]:
check_sampler(new_dl)

100%|██████████| 719/719 [00:14<00:00, 50.78it/s]
0.24695652173913044
0.7530434782608696


In [16]:
new_dl_no_replacement = DataLoader(ds, batch_size=16, 
                        sampler=WeightedRandomSampler(wl2i, len(ds)//2, replacement=False),
                        num_workers=4)

In [17]:
check_sampler(new_dl_no_replacement)

100%|██████████| 719/719 [00:26<00:00, 26.75it/s]
0.32469565217391305
0.675304347826087
